In [114]:
import pandas as pd
import numpy as np

In [383]:
evs = pd.read_parquet("events.parquet")
prs = pd.read_parquet("pairings.parquet")
pls = pd.read_parquet("placings.parquet")

## Clean up/normalise events

In [730]:
## Clean up/normalise events

cevs = evs.copy()

# There's some non-standard countries.
cevs.loc[cevs.country == "United States", "country"] = "US"
cevs.loc[cevs.country == "United Kingdom", "country"] = "GB"
cevs.loc[cevs.country == "Argentina", "country"] = "AR"

# Not *technically* correct, because of timezones, but +/- one day for dates isn't going to matter for stats
cevs["date"] = cevs.event_date.astype("datetime64[ns, UTC]").dt.date
cevs["event_id"] = cevs.event_obj_id

cols = ["date", "name", "event_id", "number_of_rounds", "total_players", "checked_in_players", "country"]

cevs = cevs[cols]

# Drop "test" events
cevs = cevs[~cevs.name.str.match(r"[tT]est")]

# Only keep events that have more than X participants
cevs = cevs[cevs.total_players >= 4].copy()

# Only big countries
# cevs = cevs[cevs.country.isin(["ES", "US", "UK"])]
# cevs = cevs[cevs.country.isin(["PL"])]

In [731]:
cevs.country.unique()

array(['ES', 'GB', 'CZ', 'US', 'UA', 'AU', 'IT', 'LV', 'PL', 'AR', 'RU',
       'FR', 'DE', 'CL', 'CA', 'PH', 'HU', 'MX', 'JP', 'ID', 'PE'],
      dtype=object)

## Clean up/normalise pairings

In [732]:
cprs = prs.copy()

# Random bogus armies
invalid_teams = ["Skittershank's Clawpack", "Servants of the Abyss"]

normal_team_changes = {
    "Adeptus Mechanicus": "Hunter Clade",
    "Adeptus Custodes": "Talons of the Emperor",
    "Astra Militarum": "Imperial Guard",
    "Kroot": "Cadre Mercenary",
    "The Gellerpox": "Gellerpox Infected",
    "Starstriders": "Elucidian Starstriders",
    "Daemons": "Chaos Daemons",
    "Deathwatch": "Space Marine",
    "Adeptus Astartes": "Space Marine",
    "T'au Empire": "Hunter Cadre",
    "Kroot Mercenaries": "Cadre Mercenary",
    "Farstalker Kinblade": "Farstalker Kinband",
    "Kroot Farstalker Kinband": "Farstalker Kinband",
    "Necrons": "Tomb World",
    "Tyranids": "Hive Fleet",
    "Ork Kommandos": "Kommando",
    "Navis Breachers": "Imperial Navy Breachers",
    "Adepta Sororitas": "Ecclesiarchy",
    "Harlequins": "Void-Dancer Troupe",
    "Drukhari": "Commorite",
    "Thousand Sons": "Warp Coven",
    "Orks": "Greenskin",
    "Genestealer Cults": "Brood Coven",
    "Asuryani": "Craftworld",
    "Heretic Astartes": "Traitor Space Marine",
}

bespoke_teams = [
    "Legionary",
    "Corsair Voidscarred",
    "Intercession Squad",
    "Pathfinders",
    "Wyrmblade",
    "Imperial Navy Breachers",
    "Gellerpox Infected",
    "Veteran Guardsmen",
    "Phobos Strike Team",
    "Void-Dancer Troupe",
    "Hunter Clade",
    "Novitiates",
    "Blooded Traitors",
    "Kommando",
    "Elucidian Starstriders",
    "Hierotek Circle",
    "Warp Coven",
    "Farstalker Kinband",
    "Kasrkin",
]

cprs = cprs[(~cprs.player1_army.isin(invalid_teams)) & (~cprs.player2_army.isin(invalid_teams))]

for p in ["player1", "player2"]:
    cprs[f"{p}_army"] = cprs[f"{p}_army"].str.strip()
    cprs[f"{p}_army"] = cprs[f"{p}_army"].replace(normal_team_changes)

cprs = cprs[cprs.event_id.isin(cevs.event_id.unique())]
cprs = cprs[cprs.player1_army.notnull() & cprs.player2_army.notnull()]

#### Add event date, and clean anything else up
cprs = cprs.merge(cevs[["date", "country", "event_id"]], on="event_id", how="left")
cprs["month"] = cprs.date.apply(lambda d: "%04d-%02d" % (d.year, d.month))

#### Fix BCP's issue where old teams get changed to bespoke ones
for p in ["player1", "player2"]:
    cprs.loc[(cprs[f"{p}_army"] == "Intercession Squad") & (cprs.month < "2022-08"), f"{p}_army"] = "Space Marine"
    cprs.loc[(cprs[f"{p}_army"] == "Void-Dancer Troupe") & (cprs.month < "2022-04"), f"{p}_army"] = "Troupe"
    cprs.loc[(cprs[f"{p}_army"] == "Warp Coven") & (cprs.month < "2022-04"), f"{p}_army"] = "Traitor Space Marine"

## Meta representation

Meta representation is defined as the number of distinct (player, competition, team) tuples. In other words: The number of times a team was chosen for tournaments.

The downside of this approach is that small tournaments can skew the meta representation, but I think this is better than the alternative where large tournaments with many rounds of play skew the results.

In [733]:
#### 1. Explode the player1/player2 representation so each player/army/tournament is a single row
p1_mrep = cprs.groupby(["event_id", "month", "player1_user_id", "player1_army"]).first().reset_index().rename(columns={"player1_user_id": "user_id", "player1_army": "army"})
p2_mrep = cprs.groupby(["event_id", "month", "player2_user_id", "player2_army"]).first().reset_index().rename(columns={"player2_user_id": "user_id", "player2_army": "army"})

mrep = pd.concat([p1_mrep, p2_mrep], ignore_index=True)[["event_id", "month", "user_id", "army"]]

#### 2. Weight by (tournament, player, army)
mrep = mrep.drop_duplicates()
mrep["is_bespoke"] = mrep.army.isin(bespoke_teams)

In [734]:
def display_meta_rep(mrr):
    mr = mrr.groupby(["month", "army"]).size().reset_index().rename(columns={0: "event_takes"})
    mr = mr.sort_values("event_takes", ascending=False)
    mr = mr.pivot(index="army", columns="month", values="event_takes").fillna(0).astype("int32")
    mr = mr.div(mr.sum(axis=0))
    mr = mr.sort_values(mr.columns[-1], ascending=False)

    return mr.style.background_gradient(cmap='Purples').format({c: '{:.2%}' for c in mr.columns})

#### All Armies

In [735]:
display_meta_rep(mrep)

month,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10
army,,,,,,,,,,
Intercession Squad,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,12.46%,14.88%
Legionary,0.00%,0.00%,0.00%,0.00%,6.98%,9.46%,13.07%,15.01%,12.75%,12.72%
Hunter Clade,2.04%,3.35%,2.51%,1.75%,3.02%,1.41%,6.45%,5.29%,5.46%,5.92%
Veteran Guardsmen,8.91%,11.73%,9.82%,10.53%,9.43%,7.85%,6.96%,6.13%,4.90%,5.49%
Kommando,11.96%,6.70%,13.01%,10.96%,9.25%,8.25%,8.15%,7.61%,8.54%,5.49%
Imperial Navy Breachers,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,1.82%,5.49%
Corsair Voidscarred,0.00%,0.00%,0.00%,0.00%,3.40%,5.84%,4.75%,5.71%,7.42%,5.06%
Blooded Traitors,0.00%,0.00%,0.00%,0.00%,0.19%,1.41%,2.04%,2.96%,1.68%,4.48%
Pathfinders,6.62%,7.08%,6.62%,8.11%,6.79%,7.44%,7.81%,6.13%,7.56%,4.48%


#### Bespoke Only

In [736]:
display_meta_rep(mrep[mrep.is_bespoke])

month,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10
army,,,,,,,,,,
Intercession Squad,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,14.66%,16.89%
Legionary,0.00%,0.00%,0.00%,0.00%,11.31%,13.28%,16.45%,19.94%,14.99%,14.43%
Hunter Clade,5.59%,7.79%,5.85%,3.77%,4.89%,1.98%,8.12%,7.02%,6.43%,6.72%
Veteran Guardsmen,24.48%,27.27%,22.87%,22.64%,15.29%,11.02%,8.76%,8.15%,5.77%,6.23%
Kommando,32.87%,15.58%,30.32%,23.58%,14.98%,11.58%,10.26%,10.11%,10.05%,6.23%
Imperial Navy Breachers,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,2.14%,6.23%
Corsair Voidscarred,0.00%,0.00%,0.00%,0.00%,5.50%,8.19%,5.98%,7.58%,8.73%,5.74%
Blooded Traitors,0.00%,0.00%,0.00%,0.00%,0.31%,1.98%,2.56%,3.93%,1.98%,5.08%
Pathfinders,18.18%,16.45%,15.43%,17.45%,11.01%,10.45%,9.83%,8.15%,8.90%,5.08%


#### Compendium Only

In [737]:
display_meta_rep(mrep[~mrep.is_bespoke])

month,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10
army,,,,,,,,,,
Space Marine,7.60%,11.44%,6.80%,13.11%,13.79%,18.18%,11.57%,27.35%,13.08%,14.63%
Hive Fleet,8.80%,13.40%,6.40%,4.92%,4.93%,6.29%,9.09%,9.40%,11.21%,12.20%
Imperial Guard,7.20%,4.25%,4.40%,4.10%,8.37%,7.69%,4.96%,5.98%,4.67%,12.20%
Talons of the Emperor,8.40%,7.52%,8.40%,9.43%,12.32%,11.89%,8.26%,11.11%,11.21%,10.98%
Death Guard,5.60%,4.58%,7.20%,5.33%,10.84%,19.58%,13.22%,11.97%,15.89%,8.54%
Cadre Mercenary,3.20%,0.65%,1.20%,1.64%,2.96%,1.40%,1.65%,0.85%,4.67%,8.54%
Grey Knights,6.00%,3.59%,10.80%,4.51%,9.85%,8.39%,11.57%,5.98%,11.21%,6.10%
Chaos Daemons,1.60%,2.94%,2.80%,3.28%,3.45%,4.90%,10.74%,6.84%,4.67%,4.88%
Commorite,7.20%,5.56%,5.20%,5.33%,1.48%,2.10%,4.96%,4.27%,1.87%,4.88%


## Win Rate

In [738]:
nm_ms = cprs[cprs.player1_army != cprs.player2_army]

p1_rs = nm_ms[["event_id", "month", "player1_army", "player1_game_result_cat"]].rename(columns={"player1_army": "army", "player1_game_result_cat": "result"})
p2_rs = nm_ms[["event_id", "month", "player2_army", "player2_game_result_cat"]].rename(columns={"player2_army": "army", "player2_game_result_cat": "result"})

w_rs = pd.concat([p1_rs, p2_rs], ignore_index=True)
w_rs["is_bespoke"] = w_rs.army.isin(bespoke_teams)

In [739]:
def display_win_rates(rs):
    rs = rs.groupby(["month", "army", "result"]).size().reset_index().rename(columns={0: "outcomes"})
    rs = rs.pivot(index=["month", "army"], columns="result", values="outcomes").fillna(0).astype("int32")
    rs["matches"] = rs.loss + rs.tie + rs.win
    
    rs = rs[rs.matches > 10]
    rs["win_rate"] = rs.win.astype("float64") / rs.matches
    rs = rs.reset_index()
    rs = rs.pivot(index="army", columns="month", values="win_rate").fillna(0)
    
    rs = rs.sort_values(rs.columns[-1], ascending=False)
    
    return rs.style.background_gradient(cmap="Blues").format({c: '{:.2%}' for c in rs.columns})

In [740]:
display_win_rates(w_rs[w_rs.is_bespoke])

month,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10
army,,,,,,,,,,
Gellerpox Infected,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,57.14%,64.71%
Imperial Navy Breachers,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,61.90%,55.08%
Void-Dancer Troupe,0.00%,0.00%,0.00%,76.67%,56.90%,59.20%,56.67%,48.89%,58.62%,53.75%
Hunter Clade,44.00%,32.76%,32.43%,25.93%,41.67%,38.10%,51.61%,48.05%,42.31%,53.19%
Blooded Traitors,0.00%,0.00%,0.00%,0.00%,0.00%,35.00%,47.22%,58.54%,58.97%,51.52%
Pathfinders,43.04%,60.61%,50.00%,54.78%,54.03%,52.99%,52.98%,58.54%,55.33%,51.38%
Legionary,0.00%,0.00%,0.00%,0.00%,53.08%,58.27%,51.79%,51.81%,49.66%,49.62%
Intercession Squad,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,55.02%,49.37%
Corsair Voidscarred,0.00%,0.00%,0.00%,0.00%,56.06%,39.18%,48.98%,50.60%,48.85%,47.71%


In [743]:
mu_p1_rs = cprs[["event_id", "month", "player1_army", "player1_game_result_cat", "player2_army"]].rename(columns={"player1_army": "army", "player2_army": "opponent", "player1_game_result_cat": "result"})
mu_p2_rs = cprs[["event_id", "month", "player2_army", "player2_game_result_cat", "player1_army"]].rename(columns={"player2_army": "army", "player1_army": "opponent", "player2_game_result_cat": "result"})

mu_rs = pd.concat([mu_p1_rs, mu_p2_rs], ignore_index=True)
mu_rs = mu_rs[mu_rs.month >= "2022-09"]

# mu_rs = mu_rs[mu_rs.army.isin(["Pathfinders", "Hunter Clade", "Intercession Squad"])]

mu_rs["both_bespoke"] = mu_rs.army.isin(bespoke_teams) & mu_rs.opponent.isin(bespoke_teams)
mu_rs = mu_rs[mu_rs.both_bespoke].groupby(["army", "opponent", "result"]).size().sort_values().reset_index()
mu_rs = mu_rs.rename(columns={0: "outcomes"})

In [744]:
mu_rs = mu_rs.pivot(index=["army", "opponent"], columns="result", values="outcomes").fillna(0).astype("int32")
mu_rs["matches"] = mu_rs.loss + mu_rs.tie + mu_rs.win
mu_rs["win_rate"] = mu_rs.win.astype("float64") / mu_rs.matches
mu_rs = mu_rs.reset_index()
mu_rs = mu_rs.pivot(index="army", columns="opponent", values="win_rate").fillna(0)


def _cc(val):
    if val == 0:
        color = "#ccc"
    elif val < 0.3:
        color = "red"
    elif val > 0.7:
        color = "green"
    else:
        color = "default"
        
    return 'color: %s' % color

mu_rs.style.format({c: '{:.0%}' for c in mu_rs.columns}).applymap(_cc)

opponent,Blooded Traitors,Corsair Voidscarred,Elucidian Starstriders,Farstalker Kinband,Gellerpox Infected,Hierotek Circle,Hunter Clade,Imperial Navy Breachers,Intercession Squad,Kasrkin,Kommando,Legionary,Novitiates,Pathfinders,Phobos Strike Team,Veteran Guardsmen,Void-Dancer Troupe,Warp Coven,Wyrmblade
army,,,,,,,,,,,,,,,,,,,
Blooded Traitors,41%,38%,100%,50%,80%,100%,50%,50%,53%,0%,40%,67%,0%,83%,0%,33%,40%,0%,67%
Corsair Voidscarred,62%,50%,67%,75%,20%,0%,50%,20%,41%,0%,39%,57%,33%,32%,78%,47%,71%,86%,36%
Elucidian Starstriders,0%,33%,0%,67%,67%,0%,33%,0%,22%,0%,67%,43%,50%,33%,0%,67%,0%,100%,100%
Farstalker Kinband,0%,25%,33%,0%,0%,0%,33%,0%,25%,0%,0%,30%,0%,50%,100%,0%,0%,100%,0%
Gellerpox Infected,20%,60%,33%,0%,50%,100%,71%,50%,62%,0%,100%,58%,89%,33%,75%,57%,80%,50%,44%
Hierotek Circle,0%,100%,0%,0%,0%,0%,0%,25%,0%,0%,0%,0%,0%,0%,0%,100%,0%,0%,25%
Hunter Clade,17%,44%,67%,33%,29%,100%,50%,38%,64%,100%,43%,43%,55%,58%,44%,47%,36%,50%,55%
Imperial Navy Breachers,17%,67%,100%,100%,38%,75%,50%,50%,33%,0%,45%,67%,86%,50%,75%,71%,0%,67%,50%
Intercession Squad,40%,48%,78%,75%,33%,100%,31%,67%,46%,100%,67%,47%,50%,36%,75%,65%,32%,42%,58%
